# KNN Imputation and Classification
With and without sklearn

In [23]:
import sklearn
from sktime.datasets import load_from_tsfile_to_dataframe
import pandas as pd
from scipy.spatial.distance import cdist
from scipy.spatial.distance import pdist

import numpy as np
from sklearn import datasets
from sklearn.impute import KNNImputer


### 1. We replace missing values in the dataset by calculating the average of their K-Nearest Neighbors,

Data load

In [24]:
X_train, Y_train=load_from_tsfile_to_dataframe("DodgerLoopGame\DodgerLoopGame_TRAIN.ts", return_separate_X_and_y=True)
X_test, Y_test=load_from_tsfile_to_dataframe("DodgerLoopGame\DodgerLoopGame_TEST.ts", return_separate_X_and_y=True)



Data processing

In [ ]:

X_train_2 = X_train[['dim_0']]

X_train_2['dim_0'] = X_train_2['dim_0'].apply(lambda x: np.array(x))
X_train_3 = np.stack(X_train_2['dim_0'].to_numpy()) #stack "puts one above the other" like rebuilding the column

X_test_full = X_test[['dim_0']]
X_test_2 = X_test_full.sample(frac=0.8, random_state=1)
X_val_2 = X_test_full.drop(X_test_2.index) #here we separate the validation X from Test

Y_val_3 = Y_test[X_val_2.index] #here we got the validation Y

X_val_2['dim_0'] = X_val_2['dim_0'].apply(lambda x: np.array(x))
X_val_3 = np.stack(X_val_2['dim_0'].to_numpy())  #here we finish processing X for validation

Y_train_3 = np.array(Y_train)

Y_test_3 = Y_test[X_test_2.index] #here we got the testing Y

X_test_2['dim_0'] = X_test_2['dim_0'].apply(lambda x: np.array(x))
X_test_3 = np.stack(X_test_2['dim_0'].to_numpy())   #here we finish processing X for testing





In [26]:


X_train_3_Nan_count=np.isnan(X_train_3).sum()
X_test_3_Nan_count=np.isnan(X_test_3).sum()
X_val_3_Nan_count=np.isnan(X_val_3).sum()

print("the dataset X_train", "has a NaN count of:", X_train_3_Nan_count)
print("the dataset X_test", "has a NaN count of:", X_test_3_Nan_count)
print("the dataset X_val", "has a NaN count of:", X_val_3_Nan_count)


the dataset X_train has a NaN count of: 65
the dataset X_test has a NaN count of: 168
the dataset X_val has a NaN count of: 104


## .2

Implement KNNImputer

In [27]:
distances_for_k=[] #we start from an empty distance K, we'll see how every K performs
for k in range(1,20):
    imputer= KNNImputer(n_neighbors=k)
    X_val_3b= X_val_3

    X_val_3b=imputer.fit_transform(X_val_3b)
    pdistances=np.sum(pdist(X_val_3b, "euclidean"))

    #pdistances=np.sum(cdist(X_val_3b, X_val_3b, "euclidean"))
    distances_for_k.append(pdistances)

print(distances_for_k)
k=(np.argmin(distances_for_k)+1) 
imputer= KNNImputer(n_neighbors=k)
X_val_4= X_val_3
X_val_4=imputer.fit_transform(X_val_4)
X_train_4=imputer.fit_transform(X_train_3)
X_test_4=imputer.fit_transform(X_test_3)

print("the optimal K for imputation is", k)

[68079.92048929445, 67924.91828385394, 67892.17623438784, 67860.97500834748, 67826.9485593052, 67825.97852645494, 67820.23292734692, 67810.70749156731, 67809.49043624672, 67804.01824031709, 67801.96125073028, 67801.70612179363, 67804.27893124646, 67803.42967927226, 67805.04860768361, 67805.84103655763, 67802.69760536042, 67802.70983963653, 67801.24019326191]
the optimal K for imputation is 19


## KNN without sklearn

Using validation data

In [28]:
np.shape(X_val_3)

(28, 288)

In [ ]:
def predictor(x_train_array, y_train_array, x_test_array, k): 
    predicted_labels=[]
    for test_row in x_test_array:
        euclidean_dist=np.empty(len(x_train_array)) #why not a list with append? lists have given me problems in previous attempts and dimentionality when turning them to numpy
        #so better to try and "initialize it" as a np array
        for i in range(len(x_train_array)):
            euclidean_dist[i]= (np.sum((test_row - x_train_array[i]) ** 2))**0.5
            #now we need to find the closest
            #according to np doccumentation, np.argsort "Returns the indices that would sort an array."
        ordered_indices=np.argsort(euclidean_dist)
        ordered_indices= ordered_indices[:k]
        y_train_labels=y_train_array[ordered_indices]
        label_options, label_options_counts = np.unique(y_train_labels, return_counts=True)
        elected_label=label_options[np.argmax(label_options_counts)]
        predicted_labels.append(elected_label)
    return np.array(predicted_labels)

print("REAL LABELS", "\n")
print(Y_val_3)

optimal_neighbors_k=1 #placeholder to initialize
accuracy=float(0.000) #placeholder to initialize
for k in range(1,17):
    predicted=predictor(X_train_3, Y_train_3, X_val_3, k)
    accuracy_loop = np.mean(predicted == Y_val_3)
    print("k", k,", acc", accuracy_loop)
    if accuracy_loop > accuracy:
        accuracy = accuracy_loop
        optimal_neighbors_k = k
print("Optimal K:", optimal_neighbors_k, ",with an accuracy of",accuracy )
predicted=predictor(X_train_3, Y_train_3, X_val_3, optimal_neighbors_k)
print("PREDICTED LABELS", "\n")

print(predicted)

REAL LABELS 

['1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '2' '2' '2' '2'
 '2' '2' '2' '2' '2' '2' '2' '2' '2' '2']
k 1 , acc 0.8214285714285714
k 2 , acc 0.7142857142857143
k 3 , acc 0.6428571428571429
k 4 , acc 0.4642857142857143
k 5 , acc 0.4642857142857143
k 6 , acc 0.4642857142857143
k 7 , acc 0.4642857142857143
k 8 , acc 0.4642857142857143
k 9 , acc 0.4642857142857143
k 10 , acc 0.42857142857142855
k 11 , acc 0.4642857142857143
k 12 , acc 0.39285714285714285
k 13 , acc 0.4642857142857143
k 14 , acc 0.35714285714285715
k 15 , acc 0.42857142857142855
k 16 , acc 0.5
Optimal K: 1 ,with an accuracy of 0.8214285714285714
PREDICTED LABELS 

['1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '2' '2' '1' '1'
 '2' '1' '2' '2' '1' '2' '2' '2' '2' '1']


Using testing data

In [30]:


print("REAL LABELS", "\n")
print(Y_test_3)

optimal_neighbors_k=1 #placeholder to initialize
accuracy=float(0.000) #placeholder to initialize
for k in range(1,17):
    predicted=predictor(X_train_3, Y_train_3, X_test_3, k)
    accuracy_loop = np.mean(predicted == Y_test_3)
    print("k", k,", acc", accuracy_loop)
    if accuracy_loop > accuracy:
        accuracy = accuracy_loop
        optimal_neighbors_k = k
print("Optimal K:", optimal_neighbors_k, ",with an accuracy of",accuracy )
predicted=predictor(X_train_3, Y_train_3, X_test_3, optimal_neighbors_k)
print("PREDICTED LABELS", "\n")

print(predicted)

REAL LABELS 

['1' '1' '2' '1' '1' '1' '2' '1' '2' '1' '2' '1' '2' '2' '1' '2' '2' '1'
 '1' '1' '1' '2' '1' '1' '1' '1' '2' '1' '2' '1' '2' '1' '2' '2' '1' '1'
 '1' '2' '2' '1' '1' '2' '1' '1' '1' '2' '1' '2' '2' '2' '2' '2' '2' '1'
 '2' '2' '1' '1' '1' '1' '2' '2' '1' '2' '2' '2' '1' '1' '1' '2' '1' '2'
 '2' '1' '1' '1' '2' '2' '2' '2' '2' '2' '2' '1' '1' '2' '1' '2' '1' '1'
 '2' '1' '1' '2' '2' '2' '1' '1' '1' '1' '2' '1' '2' '1' '2' '2' '1' '2'
 '1' '1']
k 1 , acc 0.8727272727272727
k 2 , acc 0.7181818181818181
k 3 , acc 0.7818181818181819
k 4 , acc 0.7
k 5 , acc 0.7454545454545455
k 6 , acc 0.6909090909090909
k 7 , acc 0.7090909090909091
k 8 , acc 0.6909090909090909
k 9 , acc 0.7363636363636363
k 10 , acc 0.6454545454545455
k 11 , acc 0.7272727272727273
k 12 , acc 0.6181818181818182
k 13 , acc 0.7090909090909091
k 14 , acc 0.5545454545454546
k 15 , acc 0.7
k 16 , acc 0.5272727272727272
Optimal K: 1 ,with an accuracy of 0.8727272727272727
PREDICTED LABELS 

['1' '1' '2' '1' '1' '1' 

Using training data

In [31]:

print("REAL LABELS", "\n")
print(Y_test_3)

optimal_neighbors_k=1 #placeholder to initialize
accuracy=float(0.000) #placeholder to initialize
for k in range(1,17):
    predicted=predictor(X_train_3, Y_train_3, X_train_3, k)
    accuracy_loop = np.mean(predicted == Y_train_3)
    print("k", k,", acc", accuracy_loop)
    if accuracy_loop > accuracy:
        accuracy = accuracy_loop
        optimal_neighbors_k = k
print("Optimal K:", optimal_neighbors_k, ",with an accuracy of",accuracy )
predicted=predictor(X_train_3, Y_train_3, X_train_3, optimal_neighbors_k)
print("PREDICTED LABELS", "\n")

print(predicted)

REAL LABELS 

['1' '1' '2' '1' '1' '1' '2' '1' '2' '1' '2' '1' '2' '2' '1' '2' '2' '1'
 '1' '1' '1' '2' '1' '1' '1' '1' '2' '1' '2' '1' '2' '1' '2' '2' '1' '1'
 '1' '2' '2' '1' '1' '2' '1' '1' '1' '2' '1' '2' '2' '2' '2' '2' '2' '1'
 '2' '2' '1' '1' '1' '1' '2' '2' '1' '2' '2' '2' '1' '1' '1' '2' '1' '2'
 '2' '1' '1' '1' '2' '2' '2' '2' '2' '2' '2' '1' '1' '2' '1' '2' '1' '1'
 '2' '1' '1' '2' '2' '2' '1' '1' '1' '1' '2' '1' '2' '1' '2' '2' '1' '2'
 '1' '1']
k 1 , acc 0.9
k 2 , acc 0.8
k 3 , acc 0.85
k 4 , acc 0.75
k 5 , acc 0.75
k 6 , acc 0.75
k 7 , acc 0.85
k 8 , acc 0.8
k 9 , acc 0.85
k 10 , acc 0.75
k 11 , acc 0.75
k 12 , acc 0.7
k 13 , acc 0.8
k 14 , acc 0.6
k 15 , acc 0.7
k 16 , acc 0.5
Optimal K: 1 ,with an accuracy of 0.9
PREDICTED LABELS 

['1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '2' '2' '2' '2' '2' '1' '2' '2'
 '2' '1']


# Decision Tree Regression
Decision Tree Regression model from scratch using the Residual Sum of Squares (RSS) as the split quality criterion.

In [ ]:
class TreeNode:  # we create the class with the attributes indicated, split_feature, threshold, predicted_value
  def __init__(self, predicted_value,split_feature, threshold):
    self.split_feature, self.threshold, self.predicted_value= split_feature, threshold, predicted_value
    self.right, self.left = None, None


class DecisionTreeRegressor:  #the general regressor
  def __init__(self,X, Y):
    self.X, self.Y = X, Y
  #what do I need?,---->for predict_sample I need predict---->for predict I need predict_sample--->for fit i need build_tree
  #for build_tree i need TreeNode, find_best_split---->for find_best_split I need calculate_rss first
  #therefore I should begin with calculate_rss,then find_best_split, then build_tree, then figure out both Predict functions

  def calculate_rss(self, y_left , y_right ):
    var_left, var_right= np.var(y_left), np.var(y_right)
    #Return the sum of the variances as RSS
    rss= var_left+var_right
    return rss


  def find_best_split(self, X, Y):  #this takes our X and Y, doesnt need any other parameters as we initialize rss, feature and threshold
    best_rss, best_feature, best_threshold = float('inf'), None, None
    #now for every column, we take all their values and define to which side of the threshold we are and create a list of indexes
    #that list of idexes we will pass then to get labels Y and see where they are compared to the threshold
    #For each feature in X: Iterate through unique threshold values in the feature
    #so iterate features and iterate threshold


    for feature in range(X.shape[1]): #for every column (For each feature in X)
      #print(X[:, feature])
      for threshold in X[:, feature]: #we take every value
        #will fill with 1s where it goes
        #first we initialize
        left_indices=np.zeros(X.shape[0])
        right_indices=np.zeros(X.shape[0])
        for j in range(X.shape[0]):
          if X[j, feature] < threshold:
            left_indices[j]= 1
          else:
            right_indices[j]= 1

        #now we have created two lists telling us if each row is in left or right
        #now we have the index for the x that go to the left or the right, now we take that index to redirect the Y
        left_values, right_values=Y[left_indices==1], Y[right_indices==1]
        if (len(left_values)<1 or len(right_values)<1):
          pass
        else:
          rss=self.calculate_rss(left_values, right_values)
          if rss<best_rss:
            best_rss, best_feature, best_threshold=rss, feature, threshold
    return best_rss, best_feature, best_threshold #independently of what happes, returns best rss, feature and threshold after iterating through each feature with every threshold
  

  
  def build_tree(self, X, Y, depth =0):
    #now we take a node, if we are at max depth return mean
    #then we find how are we goin to split, by calling find_best_split
    #calling this function will return the best split, which we will use for left and right
    #then all of this "subsets" will take a node and restart the process
    Node= TreeNode(predicted_value=None, split_feature=None, threshold=None)      #Attributes empty because our Fit will fill them will fill them
    #If stopping criteria are met ( min samples or max depth ): Set the node 's predicted_value to the mean of y. Return the node ."Use a stopping criterion based on either a maximum depth or a minimum number of samples in a node." Instructions ask to do either, #therefore this case will use depth
    placeholder_max_depth=10
    if depth >= placeholder_max_depth:
      Node.predicted_value=np.mean(Y)    #note for the future self: numpy mean unlike panda doesnt deal with NaN, how should i deal with them in the future?
      #print(Node.predicted_value) 
      return Node
    else:
      pass
    #Find the best feature and threshold by calling find_best_split.
    best_rss, best_feature, best_threshold= self.find_best_split(X, Y)
    if best_feature is None:
      Node.predicted_value=np.mean(Y)
      #print(Node.predicted_value) 
      return Node
  
    Node.split_feature, Node.threshold = best_feature, best_threshold #Set the node split_feature and threshold to the selected values
    #with [:, column] we take all the rows and the specified column, which means now we'll take all our input points and compare
    #the desired feature against the threshold
    left_indices, right_indices=(X[:, Node.split_feature] < Node.threshold), (X[:, Node.split_feature] >= Node.threshold)
    #now we have the indexes, we just assign them to Y
    left_values, right_values=Y[left_indices], Y[right_indices]
    X_rec_left, Y_rec_left=X[left_indices],Y[left_indices]
    X_rec_right, Y_rec_right=X[right_indices],Y[right_indices]
    #print("Node of depth", depth+1)
    #now recursivity

    Node.left= self.build_tree(X_rec_left, Y_rec_left, depth =depth+1)
    Node.right= self.build_tree(X_rec_right, Y_rec_right, depth =depth+1)
    return Node

  def fit(self, X, Y):
    self.tree= self.build_tree(X, Y, depth =0) 

  def predict(self, X):
    predictions=[]
    for sample in X:
      #print("sample",sample)
      Predicted_Sample=self.predict_sample(sample, self.tree)
      #print("p sample",Predicted_Sample)
      predictions.append(Predicted_Sample)
    predictions = np.array(predictions).reshape(-1, 1) # the instructions ask to return an array
    return predictions

  def predict_sample(self, sample, Node):
    if Node.predicted_value != None:
        return Node.predicted_value
    else:
      if sample[Node.split_feature] < Node.threshold:
        return self.predict_sample(sample, Node.left)
      else:  
        return self.predict_sample(sample, Node.right)
   



In [33]:
iris_data = pd.read_csv("iris.data", delimiter=",", header=None)

#we can replace manually, but thinking ahead this could be troublesome with a bigger list of options

labels = np.unique(iris_data.iloc[:,-1])
labels_coded = dict(zip(labels, range(1, len(labels) + 1)))
iris_data.iloc[:,-1] = iris_data.iloc[:,-1].map(labels_coded)

Y=iris_data.iloc[:, -1]
X = iris_data.drop(iris_data.columns[-1], axis=1) #axis 1 means drop column, axis 0 would be row



In [34]:

#changed the normalizing method in order to improve results, not sure if helped
#def scale(x,mean, std):
#    z= (x-mean)/std
#    return z
#for xn in X.columns:
#    mean, std= X[xn].mean(), X[xn].std()
#    X[xn] = scale(X[xn], mean, std)


def min_max_scale(x,min_x, max_x):
    z = (x-min_x)/(max_x-min_x)
    return z

for xn in X.columns:
    min_val, max_val = X[xn].min(), X[xn].max()
    X[xn] = min_max_scale(X[xn], min_val, max_val)

X_Train=X.sample(frac=0.8,  random_state=1) #i use 1 as in the documentation example
X_Test = X.drop(X_Train.index)
Y_Train = Y.loc[X_Train.index]
Y_Test = Y.drop(Y_Train.index)
X_Train=X_Train.values
Y_Train=Y_Train.values
X_Test=X_Test.values
Y_Test=Y_Test.values

In [35]:
print(Y)

0      1
1      1
2      1
3      1
4      1
      ..
145    3
146    3
147    3
148    3
149    3
Name: 4, Length: 150, dtype: object


In [36]:
DTR = DecisionTreeRegressor(X_Train, Y_Train)
DTR.fit(X_Train, Y_Train)

test_predicted=DTR.predict(X_Test)

valid_mse = np.mean((Y_Test - test_predicted)**2)
#print(test_predicted)
print("ERROR:",valid_mse)

#print(np.shape(Y_Test), np.shape(test_predicted))

for i in range(len(Y_Test)):
    print(Y_Test[i], test_predicted[i] )
    

ERROR: 0.9883748005374975
1 [1.]
1 [1.]
1 [1.]
1 [1.]
1 [1.]
1 [1.]
1 [1.]
1 [1.]
1 [1.]
1 [1.]
2 [2.]
2 [2.]
2 [2.]
2 [2.03571429]
2 [2.03571429]
2 [2.03571429]
2 [2.03571429]
2 [2.03571429]
2 [2.03571429]
2 [2.03571429]
2 [2.]
2 [2.03571429]
2 [2.03571429]
3 [2.97222222]
3 [2.97222222]
3 [2.]
3 [2.03571429]
3 [2.03571429]
3 [2.97222222]
3 [2.97222222]
